# OptiGuide Example




Here we give a simple example, as designed and illustrated in the [OptiGuide paper](https://arxiv.org/abs/2307.03875).
While the original paper is designed specifically for supply chain optimization, the general framework can be easily adapted to other applications with coding capacity.




## OptiGuide for Supply Chain Optimization: System Design Overview

The original system design for OptiGuide, tailored for supply chain optimization, is presented below.

The collaboration among three agents -- Coder, Safeguard, and Interpreter -- lies at the core of this system. They leverage a set of external tools and a large language model (LLM) to address users' questions related to supply chain applications. For a comprehensive understanding of the design and data flow, detailed information can be found in the original [paper](https://arxiv.org/abs/2307.03875).


![optiguide system](https://www.beibinli.com/docs/optiguide/optiguide_system.png)


## New Implementation



![](new_design.png)

Advantages of this multi-agent design with autogen:
- Collaborative Problem Solving: The collaboration among the user proxy agent and the assistant agents fosters a cooperative problem-solving environment. The agents can share information and knowledge, allowing them to complement each other's abilities and collectively arrive at better solutions. On the other hand, the Safeguard acts as a virtual adversarial checker, which can perform another safety check pass on the generated code.

- Modularity: The division of tasks into separate agents promotes modularity in the system. Each agent can be developed, tested, and maintained independently, simplifying the overall development process and facilitating code management.

- Memory Management: The OptiGuide agent's role in maintaining memory related to user interactions is crucial. The memory retention allows the agents to have context about a user's prior questions, making the decision-making process more informed and context-aware.



In [1]:
# Install Required Packages
# %pip install optiguide

In [2]:
# test Gurobi installation
import gurobipy as gp
from gurobipy import GRB
from eventlet.timeout import Timeout

# import auxillary packages
import requests  # for loading the example source code
import openai

# import autogen
import autogen
from autogen.agentchat import Agent, UserProxyAgent
from optiguide import OptiGuideAgent

In [3]:
config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": {
            "gpt-4",
            "gpt4",
            "gpt-4-32k",
            "gpt-4-32k-0314",
            "gpt-3.5-turbo",
            "gpt-3.5-turbo-16k",
            "gpt-3.5-turbo-0301",
            "chatgpt-35-turbo-0301",
            "gpt-35-turbo-v0301",
        }
    }
)

Now, let's import the source code (loading from URL) and also some training examples (defined as string blow).

In [4]:
# Get the source code of our coffee example
code_url = "https://raw.githubusercontent.com/microsoft/OptiGuide/main/benchmark/application/coffee.py"
response  = requests.get(code_url)


# Check if the request was successful
if response.status_code == 200:
    # Get the text content from the response
    code = response.text
else:
    raise RuntimeError("Failed to retrieve the file.")
# code = open(code_url, "r").read() # for local files


# show the first head and tail of the source code
print("\n".join(code.split("\n")[:10]))
print(".\n" * 3)
print("\n".join(code.split("\n")[-10:]))

import time
from gurobipy import GRB, Model

# Example data

capacity_in_supplier = {'supplier1': 150, 'supplier2': 50, 'supplier3': 100}

shipping_cost_from_supplier_to_roastery = {
    ('supplier1', 'roastery1'): 5,
    ('supplier1', 'roastery2'): 4,
.
.
.

# Solve
m.update()
model.optimize()

print(time.ctime())
if m.status == GRB.OPTIMAL:
    print(f'Optimal cost: {m.objVal}')
else:
    print("Not solved to optimality. Optimization status:", m.status)



In [5]:
# In-context learning examples.
example_qa = """
----------
Question: Why is it not recommended to use just one supplier for roastery 2?
Answer Code:
```python
z = m.addVars(suppliers, vtype=GRB.BINARY, name="z")
m.addConstr(sum(z[s] for s in suppliers) <= 1, "_")
for s in suppliers:
    m.addConstr(x[s,'roastery2'] <= capacity_in_supplier[s] * z[s], "_")
```

----------
Question: What if there's a 13% jump in the demand for light coffee at cafe1?
Answer Code:
```python
light_coffee_needed_for_cafe["cafe1"] = light_coffee_needed_for_cafe["cafe1"] * (1 + 13/100)
```
"""

Now, let's create an OptiGuide agent and also a user.

For the OptiGuide agent, we only allow "debug_times" to be 1, which means it can debug its answer once if it encountered errors.

In [6]:
%%capture
agent = OptiGuideAgent(
    name="OptiGuide Coffee Example",
    source_code=code,
    debug_times=1,
    example_qa="",
    llm_config={
        "seed": 42,
        "config_list": config_list,
    }
)

user = UserProxyAgent(
    "user", max_consecutive_auto_reply=0,
    human_input_mode="NEVER", code_execution_config=False
)

[autogen.oai.client: 01-03 14:46:48] {80} WARNING - openai client was provided with an empty config_list, which may not be intended.


[autogen.oai.client: 01-03 14:46:48] {80} WARNING - openai client was provided with an empty config_list, which may not be intended.


[autogen.oai.client: 01-03 14:46:48] {80} WARNING - openai client was provided with an empty config_list, which may not be intended.


Now, let's create a user's question.

In [7]:
user.initiate_chat(agent, message="What if we prohibit shipping from supplier 1 to roastery 2?")

user (to OptiGuide Coffee Example):

What if we prohibit shipping from supplier 1 to roastery 2?

--------------------------------------------------------------------------------
OptiGuide Coffee Example (to writer):


Answer Code:


--------------------------------------------------------------------------------
writer (to OptiGuide Coffee Example):

```python
model.addConstr(x[('supplier1', 'roastery2')] == 0, "no_shipping_supplier1_to_roastery2")
```

--------------------------------------------------------------------------------
OptiGuide Coffee Example (to safeguard):


--- Code ---
model.addConstr(x[('supplier1', 'roastery2')] == 0, "no_shipping_supplier1_to_roastery2")

--- One-Word Answer: SAFE or DANGER ---


--------------------------------------------------------------------------------
safeguard (to OptiGuide Coffee Example):

SAFE

--------------------------------------------------------------------------------
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread

In [8]:
user.initiate_chat(agent, message="What is the impact of supplier1 being able to supply only half the quantity at present?")

user (to OptiGuide Coffee Example):

What is the impact of supplier1 being able to supply only half the quantity at present?

--------------------------------------------------------------------------------
OptiGuide Coffee Example (to writer):


Answer Code:


--------------------------------------------------------------------------------
writer (to OptiGuide Coffee Example):

```python
# Reduce the capacity for supplier1
capacity_in_supplier['supplier1'] = 75

# Need to update the supply constraint for supplier1
for constr in model.getConstrs():
    if constr.constrName == "supply_supplier1":
        model.remove(constr)
        break

model.addConstr(
    sum(x[i] for i in shipping_cost_from_supplier_to_roastery.keys()
        if i[0] == 'supplier1') <= capacity_in_supplier['supplier1'], 
    f"supply_supplier1")

m.update()
m.optimize()
```
This code will reduce the capacity of supplier1 from 150 to 75. It finds the constraint on supplier1's capacity in the model's constraints, th

Based on the new optimization results when the capacity of supplier1 is reduced to half (from 150 to 75), the problem becomes infeasible. This means there is no solution available to meet the coffee demand of each cafe considering the quantity that each supplier can provide.

The conflicting constraints represent the points in the model where the coffee chain is unable to balance coffee supply with demand under the current conditions. In other words, the coffee chain cannot meet every cafe's coffee needs using the current suppliers' capabilities. The list of conflicting constraints includes supply constraints (quantities that each supplier can provide), conservation of flow constraints (balances between quantities supplied, roasted, and served to cafes), as well as specific coffee (light or dark) demand constraints at each cafe.

Comparing these results to the previous optimization results indicates that supplier1 has a significant impact on the current supply chain operation. When its